# Multilayer Perceptron

### Imports

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim 
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

### Settings and Dataset

In [27]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# Hyperparameters
learning_rate = 0.01
num_epochs = 5
batch_size = 64
input_size = 784
num_classes = 5


# Dataset
train_dataset = datasets.MNIST(root='data', 
                               train=True, 
                               transform=transforms.ToTensor(),
                               download=True)

test_dataset = datasets.MNIST(root='data', 
                              train=False, 
                              transform=transforms.ToTensor())


train_loader = DataLoader(dataset=train_dataset, 
                          batch_size=batch_size, 
                          shuffle=True)

test_loader = DataLoader(dataset=test_dataset, 
                         batch_size=batch_size, 
                         shuffle=False)


for images, labels in train_loader:  
    print('Image batch dimensions:', images.shape)
    print('Image label dimensions:', labels.shape)
    break

Image batch dimensions: torch.Size([64, 1, 28, 28])
Image label dimensions: torch.Size([64])


### Model

In [23]:
class MultilayerPerceptron(nn.Module):
    def __init__(self, input_size, num_classes):
        super(MultilayerPerceptron, self).__init__()
        self.fc1 = nn.Linear(input_size, 128)
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [24]:
model = MultilayerPerceptron(input_size=input_size, num_classes=num_classes)
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

In [25]:
def check_accuracy(data_loader, modal):
    if data_loader.dataset.train:
        print("Checking accuracy on training data")
    else:
        print("Checking accuracy on test data")
    correct_pred = 0
    num_examples = 0
    with torch.no_grad():
        for features, targets in data_loader:
            features = features.view(-1, 28*28).to(device)
            targets = targets.to(device)
            logits = modal(features)
            _, predicted_labels = logits.max(1)
            num_examples += targets.size(0)
            correct_pred += (predicted_labels == targets).sum()
        print(f"Got {correct_pred} / {num_examples} with accuracy {float(correct_pred)/float(num_examples)*100:.2f}")

### Train

In [28]:
for epoch in range(num_epochs):
    for batch_idx, (features, targets) in enumerate(train_loader):
        features = features.view(-1, 28*28).to(device)
        targets = targets.to(device)
       
        # forward pass
        logits = model(features)
        cost = criterion(logits, targets)
        
        # backward pass
        optimizer.zero_grad()
        cost.backward()
        
        # weight update
        optimizer.step()

        if not batch_idx % 50:
            print ('Epoch: %03d/%03d | Batch %03d/%03d | Cost: %.4f' 
                   %(epoch+1, num_epochs, batch_idx, 
                     len(train_loader), cost))

Epoch: 001/005 | Batch 000/938 | Cost: 2.1405
Epoch: 001/005 | Batch 050/938 | Cost: 2.1074
Epoch: 001/005 | Batch 100/938 | Cost: 2.0839
Epoch: 001/005 | Batch 150/938 | Cost: 2.0698
Epoch: 001/005 | Batch 200/938 | Cost: 2.0175
Epoch: 001/005 | Batch 250/938 | Cost: 1.9844
Epoch: 001/005 | Batch 300/938 | Cost: 2.0014
Epoch: 001/005 | Batch 350/938 | Cost: 1.9937
Epoch: 001/005 | Batch 400/938 | Cost: 1.9561
Epoch: 001/005 | Batch 450/938 | Cost: 2.0285
Epoch: 001/005 | Batch 500/938 | Cost: 1.9988
Epoch: 001/005 | Batch 550/938 | Cost: 1.9403
Epoch: 001/005 | Batch 600/938 | Cost: 1.9900
Epoch: 001/005 | Batch 650/938 | Cost: 1.9393
Epoch: 001/005 | Batch 700/938 | Cost: 1.9995
Epoch: 001/005 | Batch 750/938 | Cost: 1.8874
Epoch: 001/005 | Batch 800/938 | Cost: 1.8050
Epoch: 001/005 | Batch 850/938 | Cost: 1.8765
Epoch: 001/005 | Batch 900/938 | Cost: 1.8438
Epoch: 002/005 | Batch 000/938 | Cost: 1.8354
Epoch: 002/005 | Batch 050/938 | Cost: 1.7988
Epoch: 002/005 | Batch 100/938 | C

### Test

In [29]:
check_accuracy(train_loader, model)
check_accuracy(test_loader, model)

Checking accuracy on training data
Got 49698 / 60000 with accuracy 82.83
Checking accuracy on test data
Got 8353 / 10000 with accuracy 83.53
